In [ ]:
from typing import Tuple, List

import pandas as pd
import plotly.graph_objs as go

In [ ]:
FIXED_SCENARIO_RETURN = .07

In [ ]:
def get_avg_return_scenario(avg_return: float, data_key: str = "Depotwert") -> \
        List[float]:
    days_list = [(data["Datum"].iloc[i] - data["Datum"].iloc[0]).days for i in
                 range(len(data["Datum"]))]
    return [data[data_key].iloc[0] * (1 + avg_return) ** (
        (i / 365)) for i in days_list]

In [ ]:
def load_data(filename: str) -> Tuple[pd.DataFrame, float]:
    """Load data from csv file and return it as a pandas dataframe"""
    data = pd.read_csv(filename, sep=";", decimal=",", parse_dates=["Datum"],
                       date_format="%d.%m.%Y")
    avg_return = (data["Depotwert"].iloc[-1] / data["Depotwert"].iloc[0]) ** (
            365 / (data["Datum"].iloc[-1] - data["Datum"].iloc[0]).days) - 1
    data["Avg scenario"] = get_avg_return_scenario(avg_return)
    data["Fixed scenario"] = get_avg_return_scenario(FIXED_SCENARIO_RETURN)
    return data, avg_return

In [ ]:
data, avg_return = load_data("~/Desktop/Depotwerte.csv")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data["Datum"], y=data["Depotwert"], name="Depotwert"))
fig.add_trace(go.Scatter(x=data["Datum"], y=data["Depotwert"].diff(), name="Depotschwankung"))
fig.add_trace(
    go.Scatter(x=data["Datum"], y=data["Avg scenario"],
               name=f"Avg Scenario ({round(avg_return * 100, 2)}%)"))
fig.add_trace(go.Scatter(x=data["Datum"], y=data["Fixed scenario"],
                         name=f"Fixed Scenario ({round(FIXED_SCENARIO_RETURN * 100, 2)}%)"))
fig.add_trace(go.Scatter(x=data["Datum"], y=data["Net Worth"], name="Net Worth"))
fig.add_trace(go.Scatter(x=data["Datum"], y=data["Davon nicht Depot"], name="Davon nicht Depot"))
pass

In [ ]:
fig.show()